In [1]:
import matplotlib.pyplot as plt
import math as m

dbt_arr = []
cost_arr = []
pw_arr = []

for i in range(-24, 51):
    DBT = i + 273.15
    C8= -5.8002206*10**3
    C9= 1.3914993
    C10= -4.8640239*10**-2
    C11= 4.1764768*10**-5
    C12= -1.4452093*10**-8
    C13= 6.5459673
    P = 101325
    pw_mixed = m.exp(C8/DBT+C9+C10*DBT+C11*DBT**2+C12*DBT**3+C13*m.log(DBT)) 
    sh_mixed = (0.621945 * pw_mixed)/(P - pw_mixed)
    cost = 287.042 * DBT * (1+1.607858 * sh_mixed) / P
    cost_arr.append(cost)
    pw_arr.append(pw_mixed)

# Heating

In [24]:
import pandas as pd
import numpy as np
from z3 import *
import HeatingCostCalculation as cc
import statistics

#set_param('parallel.enable', True)

########################## Zone Information ###################################
num_zones = 4
sampling_time = 10                # sampling time (minute)
cp_air = 1.026
co2_fresh_air = 400
temp_fresh_air = 16.67

zone_desc = pd.read_csv('data/Zone-Description.csv')
zone_volume = zone_desc['Volume (cf)'].tolist()
zone_pp_co2 = zone_desc['CO2 Emission Per Person (cfm)'].tolist()
zone_pp_load = zone_desc['Cooling Load Per Person(kW)'].tolist()
zone_equip_load = zone_desc['Cooling Load (kW)'].tolist()
zone_max_occupant = zone_desc['Maximum Occupancy (person)'].tolist()

zone_info = pd.read_csv('data/COD.csv')
zone_occupant = [zone_info['Zone ' + str(i+1) + ' People (person)'].tolist()[0] for i in range(num_zones)]
zone_co2_setpoint = [zone_info['Zone ' + str(i+1) + ' CO2 Setpoint (ppm)'].tolist()[0] for i in range(num_zones)]
zone_temp_setpoint = [zone_info['Zone ' + str(i+1) + ' Temperature Setpoint (degree C)'].tolist()[0] for i in range(num_zones)]

########################### Cost Calculation ###############################

rh_mixed = 0.71
rh_supply = 0.45

P = 101325
CP_WATER = 4.2


dbtemp_mixed_air = [ Real( 'dbtemp_mixed_air_' + str(i)) for i in range(num_zones)]
dbtemp_supply_air = [ Real( 'dbtemp_supply_air_' + str(i)) for i in range(num_zones)]

pw_mixed_air = [ Real( 'pw_mixed_air_' + str(i)) for i in range(num_zones)]
sh_mixed_air = [ Real( 'sh_mixed_air_' + str(i)) for i in range(num_zones)]
sv_mixed_air = [ Real( 'sv_mixed_air_' + str(i)) for i in range(num_zones)]
eth_mixed_air = [ Real( 'eth_mixed_air_' + str(i)) for i in range(num_zones)]
    
temp_cond = [ Real( 'temp_cond_' + str(i)) for i in range(num_zones)]
eth_cond = [ Real( 'eth_cond_' + str(i)) for i in range(num_zones)]

pw_supply_air = [ Real( 'pw_supply_air_' + str(i)) for i in range(num_zones)]
sh_supply_air = [ Real( 'sh_supply_air_' + str(i)) for i in range(num_zones)]
eth_supply_air = [ Real( 'eth_supply_air_' + str(i)) for i in range(num_zones)]

# coil
TEMP_COIL = 6
M_COIL = 4.67

# cooling tower
temp_chil = [ Real( 'temp_chil_' + str(i)) for i in range(num_zones)]
chil_cost = [ Real( 'chil_cost_' + str(i)) for i in range(num_zones)]

# total cost
total_cost = [ Real( 'total_cost_' + str(i)) for i in range(num_zones)]


# cooling load calculation
cooling_load = [ Real( 'cooling_load' + str(i)) for i in range(num_zones)]
m_air_mixed = [ Real( 'm_air_mixed' + str(i)) for i in range(num_zones)]
m_cond = [ Real( 'm_cond' + str(i)) for i in range(num_zones)]



######################### Ventilation #########################################
v_vent_air = [ Real( 'v_vent_air_' + str(i)) for i in range(num_zones)]
v_temp_air = [ Real( 'v_temp_air_' + str(i)) for i in range(num_zones)]
v_mixed_air = [ Real( 'v_mixed_air_' + str(i)) for i in range(num_zones)]
v_fresh_air = [ Real( 'v_fresh_air_' + str(i)) for i in range(num_zones)]
v_return_air = [ Real( 'v_return_air_' + str(i)) for i in range(num_zones)]
co2_mixed_air = [ Real( 'co2_mixed_air_' + str(i)) for i in range(num_zones)]


######################### Temperature ##########################################
temp_supply_air = [ Real( 'temp_supply_air_' + str(i)) for i in range(num_zones)]
temp_mixed_air = [ Real( 'temp_mixed_air_' + str(i)) for i in range(num_zones)]



########################## Attack #############################################
att_zone_occupant = [ Real( 'att_zone_occupant_' + str(i)) for i in range(num_zones)]

att_zone_co2 = [ Real( 'att_zone_co2' + str(i)) for i in range(num_zones)]
att_zone_temp = [ Real( 'att_zone_temp' + str(i)) for i in range(num_zones)]

swap_occupant =  [ [Real( 'swap_occupant' + str(i) + '_' + str(j)) for j in range (num_zones) ] for i in range(num_zones)  ]  
bdx = [ Real( 'bdx' + str(i)) for i in range(num_zones)]


s = Solver()

for j in range(num_zones):               
    s.add(v_mixed_air[j]  == v_return_air[j] + v_fresh_air[j])

    s.add(co2_mixed_air[j] == zone_co2_setpoint[j] * (v_return_air[j] / v_mixed_air[j]) + co2_fresh_air * (v_fresh_air[j] / v_mixed_air[j]))
    s.add(temp_mixed_air[j] == zone_temp_setpoint[j] * (v_return_air[j] / v_mixed_air[j]) + temp_fresh_air * (v_fresh_air[j] / v_mixed_air[j]))


    s.add(att_zone_occupant[j] * ((zone_pp_co2[j] * 1000000 * sampling_time ) / zone_volume[j]) == (zone_co2_setpoint[j] - (( 1 - ( ( v_mixed_air[j] ) * sampling_time ) / zone_volume[j]) * zone_co2_setpoint[j] + 
                               ( v_mixed_air[j] * sampling_time * co2_mixed_air[j]) / zone_volume[j])))

    s.add(( (v_mixed_air[j]  ) * cp_air * (temp_supply_air[j] - zone_temp_setpoint[j]) ) == ( zone_equip_load[j] + att_zone_occupant[j] * zone_pp_load[j])*(0.89 * 2118.87))
    
    s.add(( (v_mixed_air[j]  ) * cp_air * (temp_supply_air[j] - att_zone_temp[j]) )  == ( zone_equip_load[j] + zone_occupant[j] * zone_pp_load[j])*(0.89 * 2118.87))


    s.add(att_zone_occupant[j] * ((zone_pp_co2[j] * 1000000 * sampling_time ) / zone_volume[j]) == 
              (zone_co2_setpoint[j] - (( 1 - ( v_vent_air[j] * sampling_time ) / zone_volume[j]) * zone_co2_setpoint[j] + 
                                   ( v_vent_air[j] * sampling_time * co2_fresh_air) / zone_volume[j])))
    
    s.add(zone_occupant[j] * ((zone_pp_co2[j] * 1000000 * sampling_time ) / zone_volume[j]) == 
              (att_zone_co2[j] - (( 1 - ( v_vent_air[j] * sampling_time ) / zone_volume[j]) * zone_co2_setpoint[j] + 
                                   ( v_vent_air[j] * sampling_time * co2_fresh_air) / zone_volume[j])))

    
    s.add((v_temp_air[j] * cp_air * 25 ) / (0.89 * 2118.87) == ( zone_equip_load[j] + att_zone_occupant[j] * zone_pp_load[j]))

    s.add(v_return_air[j] >= 0)
    s.add(v_fresh_air[j] >= 0)
    s.add(temp_supply_air[j] <= 49)

    #s.add(att_zone_occupant[j] == zone_occupant[j] + Sum(swap_occupant[j]))
    #s.add(att_zone_occupant[j] == zone_occupant[j])
    #s.add(Sum(att_zone_occupant) == sum(zone_occupant) )

    #s.add(att_zone_occupant[0] == zone_occupant[i][0])

    s.add(att_zone_occupant[j] <= zone_max_occupant[j])
    s.add(att_zone_occupant[j] >= 0)
    
    
    ################# Attack Constraints #################################
    s.add(Implies(att_zone_occupant[j] != zone_occupant[j], bdx[j] == 1))
    s.add(Implies(att_zone_occupant[j] == zone_occupant[j], bdx[j] == 0))
    
    ################ Control Constraints #################################
    s.add(Implies(v_vent_air[j] >= v_temp_air[j] , v_return_air[j] == 0))
    s.add(Implies(v_vent_air[j] < v_temp_air[j] , temp_supply_air[j] == 49))
    
    #################################### Cost Calculation #################################################

    s.add( dbtemp_mixed_air[j] == temp_mixed_air[j] + 273.15)

    #s.add( pw_mixed_air[j] == (0.05198194433*dbtemp_mixed_air[j]**3 - 41.32535618*dbtemp_mixed_air[j]**2 + 10977.13722*dbtemp_mixed_air[j] - 973835.7414) * rh_mixed )
    for k in range(-24,50):
        s.add( Implies( And(temp_mixed_air[j] >= k, temp_mixed_air[j] < k + 1)  , pw_mixed_air[j] == pw_arr[k+24] * rh_mixed) )
        s.add( Implies( And(temp_supply_air[j] >= k, temp_supply_air[j] < k + 1)  , pw_supply_air[j] == pw_arr[k+24] * rh_supply) )
        s.add( Implies( And(temp_mixed_air[j] >= k, temp_mixed_air[j] < k + 1)  , sv_mixed_air[j] == cost_arr[k+24] ))


    s.add( sh_mixed_air[j] * (P - pw_mixed_air[j]) == (0.621945 * pw_mixed_air[j]) )

    s.add(eth_mixed_air[j] == 1.006 * temp_mixed_air[j] + sh_mixed_air[j] * (2501 + 1.86* temp_mixed_air[j]))     
    #s.add(sv_mixed_air[j]  == 4.005855239*10**-7*dbtemp_mixed_air[j]**3 - 0.000318679278*dbtemp_mixed_air[j]**2 + 0.08743636124*dbtemp_mixed_air[j] - 7.490027069)

    s.add(temp_cond[j] == temp_supply_air[j])
    s.add(eth_cond[j] == CP_WATER * temp_cond[j])

    s.add(dbtemp_supply_air[j] == temp_supply_air[j] + 273.15)
    #s.add(pw_supply_air[j] == (0.05198194433*dbtemp_supply_air[j]**3 - 41.32535618*dbtemp_supply_air[j]**2 + 10977.13722*dbtemp_supply_air[j] - 973835.7414) * rh_supply )

    s.add(sh_supply_air[j] * (P - pw_supply_air[j]) == (0.621945 * pw_supply_air[j])  )        
    s.add( eth_supply_air[j] == 1.006 * temp_supply_air[j] + sh_supply_air[j] * (2501 + 1.86* temp_supply_air[j]))


    s.add(m_air_mixed[j]  == v_mixed_air[j] / (sv_mixed_air[j]* 2118.87))
    s.add(m_cond[j] == m_air_mixed[j] * (sh_mixed_air[j] - sh_supply_air[j]))
    s.add(cooling_load[j] ==  m_air_mixed[j] * (eth_supply_air[j] - eth_mixed_air[j]) + m_cond[j] * eth_cond[j])

    # chiller water input temperature = coil water output temperature
    s.add(temp_chil[j] == TEMP_COIL + (cooling_load[j] / (M_COIL * CP_WATER) ) ) 

    # chilling cost calculation (BTU/hr)
    s.add(chil_cost[j] == M_COIL * CP_WATER * (temp_chil[j] - TEMP_COIL)) 

    # total cost
    s.add(total_cost[j] == cooling_load[j] + chil_cost[j])



#s.add(Sum(v_fresh_air) > 14721.833)
#s.add(np.dot(temp_mixed_air, v_mixed_air) > 524280)
#s.add(Sum(cooling_load) > 499)
s.add(Sum(att_zone_occupant) == sum(zone_occupant) )
base_cost = 79.67


s.add(Sum(total_cost)/base_cost > 1.05)
s.add(Sum(bdx) <= num_zones)
#s.add(Or(bdx[0] != 1, bdx[1] != 1, bdx[2] != 1))
# s.add(bdx[0] == 1)
# s.add(bdx[1] == 0)
# s.add(bdx[2] == 1)
# s.add(bdx[3] == 1)

#s.add(att_zone_occupant[j] <= zone_max_occupant[j])
#s.add(att_zone_occupant[j] >= 0)

print(s.check())
    
sum_of_product = 0
sum_ = 0
tot_cost = 0
for j in range(num_zones):
    print("Actual Person", j, zone_occupant[j])
    print("Attacked Person", j, float(Fraction(str(s.model()[att_zone_occupant[j]]))))
    sum_ = sum_ + float(Fraction(str(s.model()[att_zone_occupant[j]])))

    tot_cost+= float(Fraction(str(s.model()[total_cost[j]])))
    #print("V Mixed Air", j, float(Fraction(str(s.model()[v_mixed_air[j]]))))

    #print("Temp Mixed Air", j, float(Fraction(str(s.model()[temp_mixed_air[j]]))))
    #print("Temp Supply Air", j, float(Fraction(str(s.model()[temp_supply_air[j]]))))
    #print("Cooling Load", j, float(Fraction(str(s.model()[cooling_load[j]]))) )
    #print("Total Cost: ", j, float(Fraction(str(s.model()[total_cost[j]])))) 
    #print("Pw Supply Air: ", j, float(Fraction(str(s.model()[pw_supply_air[j]])))) 
    #print('eth_mixed', float(Fraction(str(s.model()[eth_mixed_air[j]]))))
    #print('sh_supply', float(Fraction(str(s.model()[sh_supply_air[j]]))))
    #print('eth_supply', float(Fraction(str(s.model()[eth_supply_air[j]]))))

    #print('sv_mixed', sv_mixed_air[j])
    #print('sh_mixed', sh_mixed_air[j])
    #print('eth_supply', eth_supply_air[j])
    #print('sh_supply', sh_supply_air[j])
    #print("Temp Chil: ", j, float(Fraction(str(s.model()[temp_chil[j]])))) 

    #print("Vent Air", j, float(Fraction(str(s.model()[v_vent_air[j]]))))
    #print("Temp Air", j, float(Fraction(str(s.model()[v_temp_air[j]]))))
    print("Attack Zone CO2", float(Fraction(str(s.model()[att_zone_co2[j]]))))
print(sum_)
print(tot_cost)
s.model()[bdx[0]], s.model()[bdx[1]], s.model()[bdx[2]], s.model()[bdx[3]] 


# while s.check() == sat:
#     print(s.check())
#     print(s.model()[bdx[0]], s.model()[bdx[1]], s.model()[bdx[2]], s.model()[bdx[3]])
#     or_ = []
#     for j in range(num_zones):
#         or_.append(bdx[j]!=s.model()[bdx[j]])
#     s.add(Or(or_))

sat
Actual Person 0 30.0
Attacked Person 0 42.52462087674508
Attack Zone CO2 779.8041413483255
Actual Person 1 3.0
Attacked Person 1 0.34939771377922924
Attack Zone CO2 1048.3057089486238
Actual Person 2 18.0
Attacked Person 2 0.6103321663257687
Attack Zone CO2 1291.666250891309
Actual Person 3 26
Attacked Person 3 33.51564924314992
Attack Zone CO2 893.5761931018136
77.0
85.01668591111851


(1, 1, 1, 1)

In [25]:
for j in range(num_zones):
    print("V Mixed Air", j, float(Fraction(str(s.model()[v_mixed_air[j]]))))
    print("Attacked Person", j, float(Fraction(str(s.model()[att_zone_occupant[j]]))))
    print("Attack Zone CO2", float(Fraction(str(s.model()[att_zone_co2[j]]))))
    print("Attack Zone Temp", float(Fraction(str(s.model()[att_zone_temp[j]]))))

V Mixed Air 0 1566.3235356267771
Attacked Person 0 42.52462087674508
Attack Zone CO2 779.8041413483255
Attack Zone Temp 25.590220447870184
V Mixed Air 1 24.807237678325276
Attacked Person 1 0.34939771377922924
Attack Zone CO2 1048.3057089486238
Attack Zone Temp 2.966933771197183
V Mixed Air 2 37.23026214587188
Attacked Person 2 0.6103321663257687
Attack Zone CO2 1291.666250891309
Attack Zone Temp -55.325738039965486
V Mixed Air 3 1418.8291512933467
Attacked Person 3 33.51564924314992
Attack Zone CO2 893.5761931018136
Attack Zone Temp 25.16151227002446


In [21]:
print(float(Fraction(str(s.model()[temp_mixed_air[2]]))))
print(float(Fraction(str(s.model()[v_mixed_air[2]]))))
print(float(Fraction(str(s.model()[temp_supply_air[2]]))))
print(float(Fraction(str(s.model()[att_zone_temp[2]]))))

17.58625
41.09260333868424
49.0
-45.5200415699396


In [ ]:
from z3 import *
n = Real('n')
k = 
t = 
v = 
p 
s.add(n * (k * 1000000 * t ) / ) == 
              (p - (( 1 - ( q * SAMPLING_TIME ) / zone_volume[i]) * zone_co2_setpoint[i] + 
                                     ( v_mixed_air[i] * SAMPLING_TIME * co2_mixed_air[i]) / zone_volume[i])))
         